# The Battle of Neighborhoods Part 1

#### This is the final project notebook for the Applied Data Science Capstone course by IBM. The notebook will be mainly used for the capstone project..

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Assignment of week 03

### Part 01: Generating the data

In [3]:
!pip install beautifulsoup4

In [4]:
!pip install lxml

In [5]:
!pip install requests

In [6]:
from bs4 import BeautifulSoup
import requests

In [7]:
#Getting the source data from wikipedia page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [8]:
#Using BeautifulSoup4 to read the data
soup = BeautifulSoup(source, 'lxml')

In [9]:
#print(soup.prettify())

In [10]:
#Capturing the data table
table = soup.find("table", attrs={"rules":"all", "cellspacing":"0", "cellpadding":"2", "style":"width:100%; border-collapse:collapse; border:1px solid #ccc;"})

In [11]:
#Getting PostalCode and Borough Columns
PostalCode = []
Borough = []
Neighborhood = []
for i in table.find_all('td'):
    PostalCode.append(i.find('b').text)
    bor = i.find('span',style ='font-size:80%;').text
    Borough.append(bor.split('(')[0])
#Borough

In [12]:
#Cleaning and getting the Neighborhood column
Neighborhood = []
for i in table.find_all('td'):
    count = 0
    Nhd =""
    for a in i.find_all('a'):
        if count==1:
            Nhd =""
            try:
                test = a.text
                Nhd =  test 
            except Exception as e:
                Nhd = None
        elif count>1:
            try:
                test = a.text
                Nhd =  Nhd +","+test
            except Exception as e:
                Nhd = None
        count+=1
    Neighborhood.append(Nhd)
    count = 0
#Neighborhood

In [13]:
#Creating the dataframe
df = pd.DataFrame(list(zip(PostalCode, Borough, Neighborhood)), columns =['PostalCode', 'Borough', 'Neighborhood']) 

In [14]:
#Checkikng the head of the dataframe
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern,Rouge"


In [15]:
#Discarding the rows where Borough is not assigned
df = df.loc[df['Borough']!='Not assigned',:]

In [16]:
#Checking the head again
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern,Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill,Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District,Ryerson"


In [17]:
#Assigning Neighborhood as the same as Borough where Neighborhood is not present
df['Neighborhood'] = df['Neighborhood'].replace('', df['Borough']) 

In [18]:
#Shape of the dataframe
df.shape

(103, 3)

### Part 02: Adding Latitude and Longitude of the Neighborhoods the dataframe

In [20]:
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [21]:
#reading coordinate data
coord = "http://cocl.us/Geospatial_data"
coord_df = pd.read_csv(coord)

In [26]:
coord_df.head()
#coord_df.shape

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
coord_df.rename(columns={"Postal Code":"PostalCode"},inplace=True)

#Concatenating Latitude and Longitude values to the dataframe
df = pd.DataFrame.merge(df,coord_df,on='PostalCode')

In [29]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937


### Part 03: Using Geopy and Folium library to generate and explore the neighborhoods and boroughs of Toronto

In [32]:
#Using Geopy library to get Latitude and Longitude of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [33]:
# Creating a map of Neighborhoods using latitude and longitude values of Boroughs in Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers to the map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [34]:
Toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306
4,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383


In [35]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [36]:
# Creating a map of Neighborhoods using latitude and longitude values of Boroughs situated only in Downtown Toronto
map_toronto_v2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers to the map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_v2)  
    
map_toronto_v2

### Maps are not visible in the notebook published in github. So I uploaded the jpg files separately in github repository. Please check them.